# Class level relationships

## Generalization

Here, we are going to parse a whole project instead of a single python file.

In [1]:
# Clone a target project. Today, we try `httpie/cli`, a commandline HTTP client.
!git clone git@github.com:httpie/cli.git

Cloning into 'cli'...
remote: Enumerating objects: 9979, done.
remote: Counting objects: 100% (1345/1345), done.
remote: Compressing objects: 100% (169/169), done.ressing objects:  52% (88/169)
remote: Total 9979 (delta 1204), reused 1224 (delta 1173), pack-reused 8634
Receiving objects: 100% (9979/9979), 6.81 MiB | 4.16 MiB/s, done.
Resolving deltas: 100% (6837/6837), done.


In [4]:
# First, we need to read all python files in the project.
import os

path = "./cli/httpie/**/*.py"

from glob import glob

for filepath in glob(path, recursive=True):
    print(filepath)

./cli/httpie/cookies.py
./cli/httpie/sessions.py
./cli/httpie/config.py
./cli/httpie/compat.py
./cli/httpie/encoding.py
./cli/httpie/models.py
./cli/httpie/client.py
./cli/httpie/__init__.py
./cli/httpie/core.py
./cli/httpie/ssl_.py
./cli/httpie/downloads.py
./cli/httpie/context.py
./cli/httpie/utils.py
./cli/httpie/uploads.py
./cli/httpie/__main__.py
./cli/httpie/status.py
./cli/httpie/adapters.py
./cli/httpie/plugins/registry.py
./cli/httpie/plugins/__init__.py
./cli/httpie/plugins/builtin.py
./cli/httpie/plugins/manager.py
./cli/httpie/plugins/base.py
./cli/httpie/internal/daemons.py
./cli/httpie/internal/daemon_runner.py
./cli/httpie/internal/__init__.py
./cli/httpie/internal/update_warnings.py
./cli/httpie/internal/__build_channel__.py
./cli/httpie/output/streams.py
./cli/httpie/output/models.py
./cli/httpie/output/__init__.py
./cli/httpie/output/utils.py
./cli/httpie/output/writer.py
./cli/httpie/output/processing.py
./cli/httpie/output/ui/__init__.py
./cli/httpie/output/ui/palet

In [5]:
pyfiles = list(glob(path, recursive=True))
len(pyfiles)

78

## Get AST one-by-one

In [6]:
import ast

def getAST(path: str):
    with open(path, "r") as f:
        source = f.read()
        
    return ast.parse(source)

In [25]:
myast = getAST(pyfiles[0])
myast

## Detect an inheritance syntax.

In [27]:
from ast import NodeVisitor

class SuperClassNameVisitor(NodeVisitor):
    def __init__(self):
        super().__init__()
        self.names = []
    
    def visit_Name(self, node):
        self.names.append(node.id)
        return super().generic_visit(node)

class ClassVisitor(NodeVisitor):
    def __init__(self):
        super().__init__()
        self.gen_dict = {}
        
    def visit_ClassDef(self, node):
        namevisitor = SuperClassNameVisitor()
        for base in node.bases:
            namevisitor.visit(base)
        self.gen_dict[node.name] = { "parents": namevisitor.names }
#         print(node.name, list([base.id for base in node.bases]))
        return super().generic_visit(node)
        
myvisitor = ClassVisitor()
myvisitor.visit(myast)

## Store generalization relationships in a data structure

In [28]:
myvisitor = ClassVisitor()

for pyfile in pyfiles:
    myast = getAST(pyfile)
    myvisitor.visit(myast)
    
myvisitor.gen_dict

{'HTTPieCookiePolicy': {'parents': ['cookiejar']},
 'Session': {'parents': ['BaseConfigDict']},
 'ConfigFileError': {'parents': ['Exception']},
 'BaseConfigDict': {'parents': ['dict']},
 'Config': {'parents': ['BaseConfigDict']},
 'cached_property': {'parents': []},
 'HTTPMessage': {'parents': []},
 'HTTPResponse': {'parents': ['HTTPMessage']},
 'HTTPRequest': {'parents': ['HTTPMessage']},
 'RequestsMessageKind': {'parents': ['Enum']},
 'OutputOptions': {'parents': ['NamedTuple']},
 'HTTPieCertificate': {'parents': ['NamedTuple']},
 'HTTPieHTTPSAdapter': {'parents': ['HTTPAdapter']},
 'ContentRangeError': {'parents': ['ValueError']},
 'Downloader': {'parents': []},
 'DownloadStatus': {'parents': []},
 'LogLevel': {'parents': ['str', 'Enum']},
 'Environment': {'parents': []},
 'JsonDictPreservingDuplicateKeys': {'parents': ['OrderedDict']},
 'ExplicitNullAuth': {'parents': ['requests']},
 'LockFileError': {'parents': ['ValueError']},
 'ChunkedStream': {'parents': []},
 'ChunkedUploadStr

# Association

## Composition

## Aggregation